In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/boolq-dataset/train.jsonl
/kaggle/input/boolq-dataset/dev.jsonl
/kaggle/input/boolq-dataset/test.jsonl


# Command to download dataset from kaggle into colab

If you have already downloaded dataset then just upload directly inside dataset folder.
Upload the kaggle.json file in upload file command given below.


In [2]:
 ! pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d averkij/boolq-dataset

In [ ]:
!ls

In [ ]:
! mkdir dataset
! unzip boolq-dataset.zip -d dataset

You can alternatively upload dataset into a new folder named dataset.
Now we will load the dataset file for preprocessing which is present in json file format.

# Preprocessing data

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
train_df = pd.read_json("/kaggle/input/boolq-dataset/train.jsonl", lines=True)
valid_df = pd.read_json("/kaggle/input/boolq-dataset/dev.jsonl", lines=True)

In [6]:
train_df.head()

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...


In [7]:
valid_df.head()

,question,title,answer,passage
0,does ethanol take more energy make that produces,Ethanol fuel,False,All biomass goes through at least some of thes...
1,is house tax and property tax are same,Property tax,True,Property tax or 'house tax' is a local tax on ...
2,is pain experienced in a missing body part or ...,Phantom pain,True,Phantom pain sensations are described as perce...
3,is harry potter and the escape from gringotts ...,Harry Potter and the Escape from Gringotts,True,Harry Potter and the Escape from Gringotts is ...
4,is there a difference between hydroxyzine hcl ...,Hydroxyzine,True,Hydroxyzine preparations require a doctor's pr...


# Adding title to all passage and creating new feature named paragraph

In [8]:
train_df["paragraph"] = train_df["title"]+"."+train_df["passage"]

In [9]:
valid_df["paragraph"] = valid_df["title"]+"."+ valid_df["passage"]

In [10]:
train_df.head(1)

,question,title,answer,passage,paragraph
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ...","Persian language.Persian (/ˈpɜːrʒən, -ʃən/), a..."


In [11]:
valid_df.head(1)

,question,title,answer,passage,paragraph
0,does ethanol take more energy make that produces,Ethanol fuel,False,All biomass goes through at least some of thes...,Ethanol fuel.All biomass goes through at least...


In [12]:
train_df.columns

Index(['question', 'title', 'answer', 'passage', 'paragraph'], dtype='object')

In [13]:
# Since the "answer"and "passage" column are now not required we will drop it
train_df.drop(['answer', 'passage'],axis = 1,inplace = True)
valid_df.drop(['answer', 'passage'],axis = 1,inplace = True)

In [14]:
train_df.head(1)

,question,title,paragraph
0,do iran and afghanistan speak the same language,Persian language,"Persian language.Persian (/ˈpɜːrʒən, -ʃən/), a..."


In [15]:
valid_df.head(1)

,question,title,paragraph
0,does ethanol take more energy make that produces,Ethanol fuel,Ethanol fuel.All biomass goes through at least...


# Generating Answers using DistillBERT.

In [16]:
!pip install transformers

In [17]:
import transformers

In [18]:
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

In [19]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased',return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased-distilled-squad')


In [20]:
def answer_question(question, context):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    encoding = tokenizer.encode_plus(question, context)


    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    start_scores, end_scores = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    answer_start = torch.argmax(start_scores)
    end = torch.argmax(end_scores)+1
    return answer_tokens_to_string

In [21]:
def generating_answers(df):
    df["answer"] = ""
    df["position"] = 0
    for i in range(len(df)):
        
        try:
            context = df["paragraph"][i]
            question = df["question"][i]
            df["answer"][i] = answer_question(question, context)
            
        except:
            df["answer"][i]= df["title"][i] 
            print(i)
    return df

In [22]:
torch.device("cuda") # without specifying the cuda device number 

device(type='cuda')

In [23]:
train_df = generating_answers(train_df)
valid_df = generating_answers(valid_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


204
711
1430
2141
2181
2352
3547
3872
4776
5517
5618
5723
6969
7410
7574
9405
55
276
561
941
1681
1836
2150
2904
3022
3154
3190


# In case the answer return by Distillbert is empty string.

In [26]:
for i in range(len(train_df)):
    if train_df["answer"][i] == "":
        train_df["answer"][i] = train_df["title"][i]

        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
train_df.head(10)

,question,title,paragraph,answer,position
0,do iran and afghanistan speak the same language,Persian language,"Persian language.Persian (/ˈpɜːrʒən, -ʃən/), a...","It is primarily spoken in Iran , Afghanistan",0
1,do good samaritan laws protect those who help ...,Good Samaritan law,Good Samaritan law.Good Samaritan laws offer l...,Good Samaritan laws offer legal protection to ...,0
2,is windows movie maker part of windows essentials,Windows Movie Maker,Windows Movie Maker.Windows Movie Maker (forme...,Windows Movie Maker ( formerly known as Window...,0
3,is confectionary sugar the same as powdered sugar,Powdered sugar,"Powdered sugar.Powdered sugar, also called con...","Powdered sugar , also called confectioners ' s...",0
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,The Elder Scrolls Online.As with other games i...,It has a broadly similar structure to Skyrim,0
5,can you use oyster card at epsom station,Epsom railway station,Epsom railway station.Epsom railway station se...,It is not in the London Oyster card zone,0
6,will there be a season 4 of da vinci's demons,Da Vinci's Demons,Da Vinci's Demons.The series premiered in the ...,Da Vinci's Demons,0
7,is the federal court the same as the supreme c...,Federal judiciary of the United States,Federal judiciary of the United States.The fed...,The Supreme Court of the United States is the ...,0
8,did abraham lincoln write the letter in saving...,Bixby letter,Bixby letter.In the 1998 war film Saving Priva...,"In the 1998 war film Saving Private Ryan , Gen...",0
9,is batman and robin a sequel to batman forever,Batman & Robin (film),Batman & Robin (film).With the box office succ...,Batman & Robin (film),0


In [28]:
for i in range(len(valid_df)):
    if valid_df["answer"][i] == "":
        valid_df["answer"][i] = valid_df["title"][i]

        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Assigning class as specific

In [29]:
train_df["class"] = 'specific_concept_completion'
valid_df["class"] = 'specific_concept_completion'

# Finding positions of answer in paragraph

In [30]:
def position(df):
    df['answer_position'] = ""
    for i in range(len(df)):
        para = str(df["paragraph"][i]).lower()
        para = para.replace(","," ,")
        para = para.replace("."," ")
        string = str(df["answer"][i]).lower()
        position = int(para.find(string))
        if position == -1:
            df["answer"][i] = df ["title"][i]
            para = str(df["paragraph"][i]).lower()
            string = str(df["answer"][i]).lower()
            position = int(para.find(string))  
            df['answer_position'][i] = position
        else:
            df['answer_position'][i] = position
    return df

In [31]:
train_df = position(train_df)
valid_df = position(valid_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
train_df["answer_position"].value_counts()

,question,title,paragraph,answer,position,class,answer_position
0,do iran and afghanistan speak the same language,Persian language,"Persian language.Persian (/ˈpɜːrʒən, -ʃən/), a...","It is primarily spoken in Iran , Afghanistan",0,specific_concept_completion,225
1,do good samaritan laws protect those who help ...,Good Samaritan law,Good Samaritan law.Good Samaritan laws offer l...,Good Samaritan laws offer legal protection to ...,0,specific_concept_completion,19
2,is windows movie maker part of windows essentials,Windows Movie Maker,Windows Movie Maker.Windows Movie Maker (forme...,Windows Movie Maker,0,specific_concept_completion,0
3,is confectionary sugar the same as powdered sugar,Powdered sugar,"Powdered sugar.Powdered sugar, also called con...",Powdered sugar,0,specific_concept_completion,0
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,The Elder Scrolls Online.As with other games i...,It has a broadly similar structure to Skyrim,0,specific_concept_completion,303
...,...,...,...,...,...,...,...
9422,is a us district court a federal court,United States district court,United States district court.The United States...,United States district court,0,specific_concept_completion,0
9423,can a tenant get a restraining order against a...,Landlord harassment,Landlord harassment.If a landlord is found to ...,Landlord harassment,0,specific_concept_completion,0
9424,is the golden state warriors in the playoffs,Golden State Warriors,Golden State Warriors.The Warriors went into t...,Golden State Warriors,0,specific_concept_completion,0
9425,downton abbey will there be a season 7,List of Downton Abbey episodes,List of Downton Abbey episodes.Downton Abbey i...,List of Downton Abbey episodes,0,specific_concept_completion,0


# Dropping off title and position

In [33]:
train_df.drop(["title","position"],axis = 1,inplace = True)
valid_df.drop(["title","position"],axis = 1,inplace = True)

In [34]:
train_df = train_df[['paragraph','question','class','answer','answer_position']]
valid_df = valid_df[['paragraph','question','class','answer','answer_position']]

# Converting dataframe to pickle

In [35]:
import pickle


In [36]:
valid_pickle = valid_df.to_dict(orient='records')
train_pickle = train_df.to_dict(orient='records')

In [37]:
output = open('instances_corefs_dev.pickle', 'wb')
pickle.dump(valid_pickle, output)
output1 = open('instances_corefs_train.pickle', 'wb')
pickle.dump(train_pickle, output1)